<h2 style="text-align:center;">Data Preprocessing</h2>

<h2>Imports</h2>

In [1]:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import os
import sys
import pandas as pd

<h2>🔃Loading Dataset</h2>

In [2]:

# Detect project root by going up until we find the 'src' directory
current_dir = os.getcwd()
while not os.path.isdir(os.path.join(current_dir, 'src')):
    current_dir = os.path.dirname(current_dir)
    if current_dir == os.path.dirname(current_dir):  # Reached filesystem root
        raise FileNotFoundError("Could not find 'src' directory in any parent folders.")

# Set project root and add it to sys.path
PROJECT_ROOT = current_dir
print(f"Setting project root: {PROJECT_ROOT}")
os.chdir(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)


from src.data import loader, preprocessor
from src.visualization import exploration_visualized


Setting project root: c:\Users\HP\Desktop\Healthcare_test_results_classification-


In [3]:

project_root = r'C:\Users\HP\Desktop\Healthcare_test_results_classification-'  # Replace with the actual path
data_path = os.path.join(project_root, 'data', 'raw')

train_df, test_df = loader.load_data(
    train_path=os.path.join(data_path, 'train data.csv'),
    test_path=os.path.join(data_path, 'test data.csv')
)

train_df.head()


,ID,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,1,Bobby JacksOn,27,Female,O-,Asthma,06/06/2022,Mark Hartman Jr.,Sons and Miller,Cigna,2625.980554,379,Elective,18/08/2022,Ibuprofen,Normal
1,2,LesLie TErRy,68,Female,O-,Cancer,19/11/2021,Angela Contreras,White-White,Cigna,1471.387317,113,Elective,20/11/2021,Ibuprofen,Inconclusive
2,3,DaNnY sMitH,21,Female,A+,Hypertension,05/03/2022,David Ruiz,Group Middleton,Medicare,5131.488104,154,Emergency,16/05/2022,Paracetamol,Normal
3,4,andrEw waTtS,91,Male,AB-,Diabetes,06/04/2020,Jenny Griffith,Morris-Arellano,Blue Cross,8972.793157,293,Urgent,26/04/2020,Ibuprofen,Abnormal
4,5,adrIENNE bEll,52,Female,A+,Diabetes,31/12/2022,Cynthia Scott,Williams-Davis,Blue Cross,2015.522684,265,Emergency,11/02/2023,Penicillin,Abnormal


<h2>Handling DateTime Datatype</h2>

In [4]:

class DateFeatureHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return preprocessor.handle_date_features(X)

In [5]:

# fixed_datatypes=preprocessor.handle_date_features(train_df)
# preprocessor.save_processed_df(fixed_datatypes,"processed_train_data.csv",output_dir="data/processed")
# fixed_datatypes.head()


<h2>Handling Missing Values</h2>

In [6]:

class MissingValueImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return preprocessor.handle_missing_values(X)

In [7]:
# handled_missing=preprocessor.handle_missing_values(fixed_datatypes)
# preprocessor.save_processed_df(handled_missing,"processed_train_data.csv",output_dir="data/processed")


<h2>Encoding</h2>

In [8]:
class FeatureEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return preprocessor.encoding_features(X)

In [9]:

# # encoded_df = preprocessor.encode_students_dataset(train_df)
# # print("Encoded shape:", encoded_df.shape)
# encoded_df=preprocessor.encoding_features(handled_missing)

# preprocessor.save_processed_df(encoded_df,"processed_train_data.csv",output_dir="data/processed")
# encoded_df.head()

<h2>Scaling (Standrization)</h2>


PCA, Logistic Regression, SVM, MLP → Use StandardScaler

In [10]:

class FeatureScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return preprocessor.scale_numerical_features(X)


In [11]:
# scaled_df=preprocessor.scale_numerical_features(encoded_df)

# preprocessor.save_processed_df(scaled_df,"processed_train_data.csv",output_dir="data/processed")
# scaled_df.head()

<h2>Creating & Running The Pipeline</h2>

In [12]:
preprocessing_pipeline = Pipeline([
    ('imputer', MissingValueImputer()),
    ('date_handler', DateFeatureHandler()),
    ('encoder', FeatureEncoder()),
    ('scaler', FeatureScaler())
])

# df = pd.read_csv("data/raw/train data.csv")
processed_df = preprocessing_pipeline.fit_transform(train_df)

# Optional: Save result
preprocessor.save_processed_df(processed_df, filename="preprocessed_train_data.csv")


✅Missing values handled:
Filled categorical column 'Blood Type' with mode: B-
Filled categorical column 'Doctor' with mode: Angela Contreras
Filled categorical column 'Hospital' with mode: Houston PLC
Filled categorical column 'Insurance Provider' with mode: Blue Cross
Filled numerical column 'Billing Amount' with median: 5313.5078885
Filled categorical column 'Admission Type' with mode: Urgent

📅 Detected and converted date columns: ['Date of Admission', 'Discharge Date']
✅ Label encoded 'Gender'.
🎯 Label encoded target column 'Test Results'.
✅ One-hot encoded 'Blood Type'.
✅ One-hot encoded 'Medical Condition'.
✅ One-hot encoded 'Insurance Provider'.
✅ One-hot encoded 'Admission Type'.
✅ One-hot encoded 'Medication'.
✅ Frequency encoded 'Doctor'.
✅ Frequency encoded 'Hospital'.

📐 Encoded shape: (50000, 30)
✅ Scaled numerical columns: ['Age', 'Billing Amount']

📐 Scaled shape: (50000, 30)
✅ Saved processed DataFrame to:
c:\Users\HP\Desktop\Healthcare_test_results_classification-\data